In [8]:
import os
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from typing import List, Optional, Tuple


In [9]:
import a_env_vars
import os
os.environ["OPENAI_API_KEY"] = a_env_vars.OPENAI_API_KEY


In [10]:
# Define a new Pydantic model with field descriptions and tailored for Twitter.
class Command(BaseModel):
    object: str = Field(description="Objeto al que se aplica la instruccion: campo, columna, fila, filtro")
    field: str = Field(description="Campo de la base al que aplica")
    function: Optional[str]  = Field(description="Funcion que se aplica al object de tipo campo puede ser SUM, MAX, MIN,AVG ")
    operator:  Optional[str] = Field(description="Operador que se aplica si es un Filtro puede ser: >,<,=,<>, in")
    value: Optional[str]  = Field(description="Valor de la condicion del filtro")
    value_to: Optional[str]  = Field(description="Valor para el operador in, valor desde")
    value_from: Optional[str]  = Field(description="Valor para el operador in, valor hasta")

class Report(BaseModel):
    type: str = Field(description="Tipo de reporte: Grilla, Lista agrupados, dato")
    description: str = Field(description="Nombre del reporte")
    commands: List[Command]
    

In [11]:
# Instantiate the parser with the new model.
parser = PydanticOutputParser(pydantic_object=Report)

# Update the prompt to match the new query and desired format.
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            """Completa las instrucciones para llenar un reporte, el reporte se construye ejecutando una lista de instrucciones
            por ejemplo: para la grilla de totaizar los vuelos por aeropuesrtos de origen y destino en 2014, las sigueintes instrucciones:
            type= grilla, description= Lista de viajes, command=[
                [object=columna, field=aeropuerto_destino] (agrega columna con aeropuerto destino)
                [object=fila, field=aeropuerto_origen] (agrega fila con aeropuerto origen)
                [object=campo, field=tarifafactura, function=SUM] (agrega campo acumulado de tarifa)
                [object=filtro, field=fechadespegue,operator=in,value_to=01/01/2014,value_from=31/12/2014] (agrega filtro en el año 2014)
            ]
            \n{format_instructions}\n{question}"""
        )
    ],
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions(),
    },
)

In [12]:
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    max_tokens=1000
)

# Generate the input using the updated prompt.
user_query = (
    """"
  Dado los siguientes campos en una tabla de boletos de avion


    interface_id: id unico
    route: ruta con el formato SEGMENTO|SEGMENTO|.. y un SEGMENTO=Aeropuerto origen(operadora vuelo)Aeropuerto Destino, ejemplo: UIO(EQ)CCS|CCS(EQ)UIO 
    nro_iata: número identificador agencia de viaje
    gds: id de sistema global de venta de pasajes
    codigopnr: codigo de reserva
    numeroboleto: numero de boleto
    boleto_original: numero de boleto original
    
	void: anulado pueder ser N (activo) o S (Anulado)
    void_date: fecha de anulacion si fue anulado
	tipo_pasajero: Tipo de pasajero puede ser ADT (adulto) INF (Bebes) CHD (Niños)
    nombre_pasajero: Apeelido y nombre del pasajero
    creation_date: fecha de venta del boleto
    NumeroPasajero: cantidad de pasajeros en la reserva
	departure_date: fecha de despegue
	dias_viajados: dias de duración del viaje
    arrive_date: fecha de arribo
    dias_compra: dias desde la compra hasta el despegue

    pais: código de pais donde se vendió el boleto
    tipoboleto: Tipo de boleto K(Electronic ticket) T (Manual)
    
	aeropuerto_destino: codigo iata de aeropuerto destino
    pais_destino: código de país destino
   
	aeropuerto_origen: codigo iata de aeropuerto destino
    pais_origen:código de país origen

    codigoaerolinea: código de aerolinea del boleto
    internacional: vuelo puede ser N (internacional) o S (dometico)
    internacional_descr: Descripción tipo de vuelo: International o Domestic
    codigo_aerolinea_intern: Codigo de aerolinea operadora del vuelo,

    codigo_base_moneda: codigo de moneda base
	codigo_moneda: codigo de moneda del boleto

    monto_tarjeta: monto pagado con tarjeta en moneda del boleto
  
    filtrar por volados en este cuatrimestre 
      completar el json
    """
)
_input = prompt.format_prompt(question=user_query)

output = chat_model(_input.to_messages())
parsed = parser.parse(output.content)
print(output.content)
print(parsed)

{
    "type": "grilla",
    "description": "Reporte de vuelos en el cuatrimestre",
    "commands": [
        {
            "object": "columna",
            "field": "aeropuerto_destino"
        },
        {
            "object": "fila",
            "field": "aeropuerto_origen"
        },
        {
            "object": "campo",
            "field": "monto_tarjeta",
            "function": "SUM"
        },
        {
            "object": "filtro",
            "field": "departure_date",
            "operator": "in",
            "value_to": "01/01/2022",
            "value_from": "30/04/2022"
        }
    ]
}
type='grilla' description='Reporte de vuelos en el cuatrimestre' commands=[Command(object='columna', field='aeropuerto_destino', function=None, operator=None, value=None, value_to=None, value_from=None), Command(object='fila', field='aeropuerto_origen', function=None, operator=None, value=None, value_to=None, value_from=None), Command(object='campo', field='monto_tarjeta', function=